In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize, differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

import itertools
from collections import Counter

In [6]:
potential = 'DW'
cutoff = 8
shots = 1024

In [7]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [8]:
num_qubits

4

In [9]:
min_3_ev = eigenvalues.argsort()[:3]
min_eigenvector = np.asarray(eigenvectors[:, min_3_ev[1]])

In [10]:
min_eigenvalue

np.complex128(0.8845804438664059+0j)

In [13]:
operator_pool = []
p = 0.0
for i in range(num_qubits):
    operator_pool.append(qml.RY(p,wires=[i]))
    operator_pool.append(qml.RZ(p,wires=[i]))
    operator_pool.append(qml.RX(p,wires=[i]))


In [14]:
c_pool = []

for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                c_pool.append(qml.CZ(wires=[control, target]))
                c_pool.append(qml.CNOT(wires=[control, target]))

In [15]:
operator_pool = operator_pool + c_pool

In [16]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
@qml.qnode(dev)
def circuit(params, op_list):

    basis = [0,0,0,0]
    qml.BasisState(basis, wires=range(num_qubits))

    param_index = 0
    for op in op_list:
        o = type(op)
        if (o == qml.CNOT) or (o == qml.CZ):
            o(wires=op.wires)
        else:
            o(params[param_index], wires=op.wires)
            param_index +=1


    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [17]:
dev2 = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev2)
def grad_circuit(param, operator_ham, op_list, op_params):

    basis = [0,0,0,0]
    qml.BasisState(basis, wires=range(num_qubits))

    param_index = 0
    for op in op_list:
        o = type(op)
        if (o == qml.CNOT) or (o == qml.CZ):
            o(wires=op.wires)
        else:
            o(op_params[param_index], wires=op.wires)
            param_index +=1

    oph = type(operator_ham)
    if (oph == qml.CNOT) or (oph == qml.CZ):
        oph(wires=operator_ham.wires)
    else:
        oph(param, wires=operator_ham.wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [18]:
def compute_grad(param, operator, op_list, op_params):
    params = pnp.tensor(param, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(params, operator, op_list, op_params)
    return grad

In [19]:
def cost_function(times, op_list):

    times = pnp.tensor(times, requires_grad=True)
    energy = circuit(times, op_list)
    
    return energy

In [21]:
operator_pool

[RY(0.0, wires=[0]),
 RZ(0.0, wires=[0]),
 RX(0.0, wires=[0]),
 RY(0.0, wires=[1]),
 RZ(0.0, wires=[1]),
 RX(0.0, wires=[1]),
 RY(0.0, wires=[2]),
 RZ(0.0, wires=[2]),
 RX(0.0, wires=[2]),
 RY(0.0, wires=[3]),
 RZ(0.0, wires=[3]),
 RX(0.0, wires=[3]),
 CZ(wires=[0, 1]),
 CNOT(wires=[0, 1]),
 CZ(wires=[0, 2]),
 CNOT(wires=[0, 2]),
 CZ(wires=[0, 3]),
 CNOT(wires=[0, 3]),
 CZ(wires=[1, 0]),
 CNOT(wires=[1, 0]),
 CZ(wires=[1, 2]),
 CNOT(wires=[1, 2]),
 CZ(wires=[1, 3]),
 CNOT(wires=[1, 3]),
 CZ(wires=[2, 0]),
 CNOT(wires=[2, 0]),
 CZ(wires=[2, 1]),
 CNOT(wires=[2, 1]),
 CZ(wires=[2, 3]),
 CNOT(wires=[2, 3]),
 CZ(wires=[3, 0]),
 CNOT(wires=[3, 0]),
 CZ(wires=[3, 1]),
 CNOT(wires=[3, 1]),
 CZ(wires=[3, 2]),
 CNOT(wires=[3, 2])]

In [28]:
op_list = []
op_params = []
grad_list = []
param=0.0
for op in operator_pool:
    print(op)
    grad = compute_grad(param, op, op_list, op_params)
    o=type(op)
    if (o == qml.CNOT) or (o == qml.CZ):
        grad_op = o(wires=op.wires)
    else:
        grad_op = o(param, wires=op.wires)
    grad_list.append((grad_op,abs(grad)))

RY(0.0, wires=[0])
RZ(0.0, wires=[0])
RX(0.0, wires=[0])
RY(0.0, wires=[1])
RZ(0.0, wires=[1])
RX(0.0, wires=[1])
RY(0.0, wires=[2])
RZ(0.0, wires=[2])
RX(0.0, wires=[2])
RY(0.0, wires=[3])
RZ(0.0, wires=[3])
RX(0.0, wires=[3])
CZ(wires=[0, 1])
CNOT(wires=[0, 1])
CZ(wires=[0, 2])
CNOT(wires=[0, 2])
CZ(wires=[0, 3])
CNOT(wires=[0, 3])
CZ(wires=[1, 0])
CNOT(wires=[1, 0])
CZ(wires=[1, 2])
CNOT(wires=[1, 2])
CZ(wires=[1, 3])
CNOT(wires=[1, 3])
CZ(wires=[2, 0])
CNOT(wires=[2, 0])
CZ(wires=[2, 1])
CNOT(wires=[2, 1])
CZ(wires=[2, 3])
CNOT(wires=[2, 3])
CZ(wires=[3, 0])
CNOT(wires=[3, 0])
CZ(wires=[3, 1])
CNOT(wires=[3, 1])
CZ(wires=[3, 2])
CNOT(wires=[3, 2])


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\autograd\tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [27]:
grad_list


[(pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  tensor(0.09317364, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RZ,
  tensor(2.05667035e-17, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RX,
  tensor(0.09317364, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  tensor(2.74463991, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RZ,
  tensor(2.05667035e-17, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RX,
  tensor(2.14299383, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  tensor(2.52877862, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RZ,
  tensor(2.05667035e-17, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RX,
  tensor(0.79197598, requires_grad=True)),
 (pennylane.ops.qubit.parametric_ops_single_qubit.RY,
  tensor(2.75763145, requires_grad=True)),
 (pennylane.ops.qu

In [30]:
num_steps = 4
op_list = []
op_params = []

#variables
max_iter = 5000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 10

number_grad_checks = 1

for i in range(num_steps):

    print("########################################")
    print(f"step: {i}")

    max_ops_list = []

    for param in np.random.uniform(0.0, 0.0, size=number_grad_checks):
        grad_list = []
        for op in operator_pool:
            grad = compute_grad(param, op, op_list, op_params)
            o=type(op)

            if (o == qml.CNOT) or (o == qml.CZ):
                grad_op = o(wires=op.wires)
            else:
                grad_op = o(param, wires=op.wires)

            grad_list.append((grad_op,abs(grad)))

        max_op, max_grad = max(grad_list, key=lambda x: x[1])
        #print(f"For param {param} the max op is {max_op} with grad {max_grad}")
        max_ops_list.append(max_op)

    counter = Counter(max_ops_list)
    most_common_gate, count = counter.most_common(1)[0]
    print(f"Most common gate is {most_common_gate}")
    op_list.append(most_common_gate)

    # Generate Halton sequence
    num_dimensions = len(op_list) + 1
    num_samples = popsize
    halton_sampler = Halton(d=num_dimensions)
    halton_samples = halton_sampler.random(n=num_samples)
    scaled_samples = 2 * np.pi * halton_samples

    bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
    x0 = np.concatenate((op_params, np.array([np.random.random()*2*np.pi])))
    
    res = differential_evolution(cost_function,
                                    bounds,
                                    x0=x0,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )
    
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break
        elif min_e >= pre_min_e: 
            print("Energy increased")
            op_list.pop()
            op_params = pre_op_params
            break


    

########################################
step: 0
Most common gate is RY(np.float64(0.0), wires=[3])


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\autograd\tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Min E: 0.9515526242302892
False
########################################
step: 1
Most common gate is RY(np.float64(0.0), wires=[3])
Min E: 0.9565056762083577
False
Energy increased


In [ ]:
op_list

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits, shots=2)
@qml.qnode(dev)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CNOT:
            qml.CNOT(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.state()

In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

In [ ]:
def overlap_function(params):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [ ]:
bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]

# Generate Halton sequence
num_dimensions = len(op_list)
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-6
abs_tol = 1e-5
popsize = 20

print("Running for overlap")
overlap_res = differential_evolution(overlap_function,
                                    bounds,
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy,
                                    popsize=popsize,
                                    init=scaled_samples,
                                    )

In [ ]:
overlap_res.fun

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def energy_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.CNOT:
            qml.CNOT(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [ ]:
x0 = overlap_res.x
#x0 = hf_res.x
energy_circuit(x0)

In [ ]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [ ]:
data

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [ ]:
#variables
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
atol = 1e-3
popsize = 20

num_steps = 5
op_list = []
op_params = []

for i in range(num_steps):

    print(f"step: {i}")

    grad_list = []

    for op in operator_pool:
        grad = compute_grad(op, op_list, op_params)
        grad_list.append(abs(grad))

    maxidx = np.argmax(grad_list)
    op_list.append(operator_pool[maxidx])

    bounds = [(0, 2 * np.pi) for _ in range(len(op_list))]
    res = differential_evolution(cost_function,
                                    bounds,
                                    args=(op_list,),
                                    maxiter=max_iter,
                                    tol=tol,
                                    atol=atol,
                                    strategy=strategy,
                                    popsize=popsize
                                    )
    if i!=0: pre_min_e = min_e
    min_e = res.fun
    pre_op_params = op_params
    op_params = res.x

    print(f"Min E: {min_e}")
    print(res.success)

    print("Testing CZ pool")
    cz_e = []
    for term in cz_pool:
        energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
        cz_e.append(energy)

    min_cz_e = cz_e[np.argmin(cz_e)]
    min_cz_term = cz_pool[np.argmin(cz_e)]
    if min_cz_e < min_e:
        print(f"Adding {min_cz_term} reduces energy further")
        op_list.append(min_cz_term)
        min_e = min_cz_e
        print(f"Min E: {min_e}")
    
    if i!=0:
        if abs(pre_min_e - min_e) < 1e-8:
            print("gradient converged")
            op_list.pop()
            op_params = pre_op_params
            break


    